In [4]:
import pandas as pd
%load_ext autoreload
%autoreload 2
    
import sys, os
sys.path.append(os.path.abspath(".."))
import cga_utils
import prompt_versions


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.s2_1.db"))

In [6]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="qwen3:4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0)  # vagy bármely más elérhető Ollama model
#llm = ChatOllama(model="hf.co/unsloth/Qwen3-4B-Instruct-2507-GGUF:latest")  # vagy bármely más elérhető Ollama modell

In [7]:
dataset_tr = pd.read_json('../dataset_raw/tatqa_dataset_train.json')    
dataset_t = pd.read_json('../dataset_raw/tatqa_dataset_test.json')    
dataset_tg = pd.read_json('../dataset_raw/tatqa_dataset_test_gold.json')    
dataset_d = pd.read_json('../dataset_raw/tatqa_dataset_dev.json')    

sample_df=dataset_tg
#sample_df = dataset_tg.sample(frac=0.20, random_state=42)
#sample_df = dataset_tr.sample(frac=0.031, random_state=42)


In [8]:
len(dataset_tr), len(dataset_d),  len(dataset_t), len(dataset_tg), len(sample_df)

(2201, 278, 278, 277, 277)

In [18]:
for i, item in sample_df.iterrows():    
    table = item['table']['table']     
    for q_item in item['questions']:
        for idev, itemd in dataset_d.iterrows():
            tabled = itemd['table']['table']     
            for q_itemd in itemd['questions']: 
                if q_itemd['answer_type'] == "arithmetic" and q_item['question'] == q_itemd['question']:
                #if q_item['question'] == q_itemd['question']:
                    print('duplicate', q_item['question'], q_itemd['question'], q_item['answer'],  q_itemd['answer'])
                

duplicate What was the percentage change in total revenue between 2018 and 2019? What was the percentage change in total revenue between 2018 and 2019? 10.98 5.22
duplicate What was the percentage change in total revenue between 2018 and 2019? What was the percentage change in total revenue between 2018 and 2019? 10.98 5.22
duplicate What is the percentage change in revenues from 2018 to 2019? What is the percentage change in revenues from 2018 to 2019? 13.48 1.05
duplicate What is the average voyage revenue in 2018 and 2019? What is the average voyage revenue in 2018 and 2019? 303118 303118


# V18

In [ ]:
import pandas as pd
messages = prompt_versions.prompt_versions['V18'].copy() #Base
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

messages2 = prompt_versions.prompt_versions['V18'].copy() #Base
messages2 = [(a, t.replace('/no_think','')) for (a,t)  in messages2]
messages2[-1] = ( messages2[-1][0],  messages2[-1][1] + " /no_think" ) 

messages3 = prompt_versions.prompt_versions['V18'].copy() #Base
messages3 = [(a, t.replace('/no_think','')) for (a,t)  in messages3]
messages3[-1] = ( messages3[-1][0],  messages3[-1][1] + "  /no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

In [15]:
annotated_results = cga_utils.annotate_results(res)
annotated_results.to_csv('../res/q3_v18_test.csv')
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  invalid syntax (<unknown>, line 4)
Annotaion warning:  invalid syntax (<unknown>, line 18)


0.5668789808917197

# V29

In [28]:
import pandas as pd
messages = prompt_versions.prompt_versions['V29'].copy() #Base
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

messages2 = prompt_versions.prompt_versions['V29'].copy() #Base
messages2 = [(a, t.replace('/no_think','')) for (a,t)  in messages2]
messages2[-1] = ( messages2[-1][0],  messages2[-1][1] + " /no_think" ) 

messages3 = prompt_versions.prompt_versions['V29'].copy() #Base
messages3 = [(a, t.replace('/no_think','')) for (a,t)  in messages3]
messages3[-1] = ( messages3[-1][0],  messages3[-1][1] + "  /no_think" )

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771693397.2076898


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  0.004s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03  0.003s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1219725.0, 1112987.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 0.003s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 0.003s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000b

In [19]:
annotated_results = cga_utils.annotate_results(res)
annotated_results.to_csv('../res/q3_v29_test.csv')
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  cannot assign to operator (<unknown>, line 4)


0.6645435244161358

In [22]:
res = cga_utils.execute_dataset_predictions(llm, messages2, dataset=sample_df)
annotated_results = cga_utils.annotate_results(res)
#annotated_results.to_csv('../res/q3_v29_test.csv')
cga_utils.calc_overall_em(annotated_results)

1771689220.6111636


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  5.120s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03  1.705s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1219725.0, 1112987.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 1.051s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 1.380s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000b

0.6666666666666666

In [30]:
res = cga_utils.execute_dataset_predictions(llm, messages3, dataset=sample_df)
annotated_results = cga_utils.annotate_results(res)
#annotated_results.to_csv('../res/q3_v29_test.csv')
cga_utils.calc_overall_em(annotated_results)

1771699519.3825798


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  0.008s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? Success: -9.03 0.003s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1042791.0, 948578.0, -9.034696310190633] || [948578.0, 1042791.0, 1042791.0] || True >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 0.003s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 0.003s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000baafcb
What was the aver

0.6709129511677282

# V23

In [ ]:
import pandas as pd
messages = prompt_versions.prompt_versions['V23']
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

In [24]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  invalid syntax (<unknown>, line 20)


0.6475583864118896

# V41

In [31]:
import pandas as pd

messages = prompt_versions.prompt_versions['V41'].copy() #Base
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

messages2 = prompt_versions.prompt_versions['V41'].copy() #Base
messages2 = [(a, t.replace('/no_think','')) for (a,t)  in messages2]
messages2[-1] = ( messages2[-1][0],  messages2[-1][1] + " /no_think" ) 

messages3 = prompt_versions.prompt_versions['V41'].copy() #Base
messages3 = [(a, t.replace('/no_think','')) for (a,t)  in messages3]
messages3[-1] = ( messages3[-1][0],  messages3[-1][1] + "  /no_think" )

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1772067437.0610743


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  0.008s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03  0.007s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1112987.0, 1219725.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 0.003s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 0.003s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000b

In [32]:
annotated_results = cga_utils.annotate_results(res)
annotated_results.to_csv('../res/q3_v41_test.csv')
cga_utils.calc_overall_em(annotated_results)

0.6709129511677282

In [25]:
res = cga_utils.execute_dataset_predictions(llm, messages2, dataset=sample_df)
annotated_results = cga_utils.annotate_results(res)
#annotated_results.to_csv('../res/q3_v29_test.csv')
cga_utils.calc_overall_em(annotated_results)

1771690832.9934113


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  1.479s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03  1.487s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1112987.0, 1219725.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 1.066s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 1.393s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000b

0.6687898089171974

In [26]:
res = cga_utils.execute_dataset_predictions(llm, messages3, dataset=sample_df)
annotated_results = cga_utils.annotate_results(res)
#annotated_results.to_csv('../res/q3_v29_test.csv')
cga_utils.calc_overall_em(annotated_results)

1771691605.0920577


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  1.274s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03  1.317s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1112987.0, 1219725.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 1.067s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 1.382s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000b

0.6794055201698513

# V43c

In [27]:
import pandas as pd
messages = prompt_versions.prompt_versions['V43c']
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771018372.747808


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5 
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03 
<< ((948,578-1,042,791)/1,042,791) * 100 || [1112987.0, 1219725.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? failure: 643.0 good answer:  643 
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000baafcb
Wha

In [28]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  cannot assign to operator (<unknown>, line 4)


0.6687898089171974

# V31

In [29]:
import pandas as pd
messages = prompt_versions.prompt_versions['V31']
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771018376.0645576


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5 
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -10.53 good answer:  -9.03 
<< ((948,578-1,042,791)/1,042,791) * 100 || [1122729.0, 1004450.0, -10.534955452295256] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? failure: 643.0 good answer:  643 
<< 17,845-17,202 || [17845.0, 17202.0, 643.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0
<< 1,320 - 1,086 || [1086.0, 1320.0, 234.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd3

In [30]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  invalid syntax (<unknown>, line 1729)
Annotaion warning:  cannot assign to operator (<unknown>, line 4)


0.6518046709129511

In [31]:
import pandas as pd
messages = prompt_versions.prompt_versions['V44b']
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771018379.4455855


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5 
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -9.03 good answer:  -9.03 
<< ((948,578-1,042,791)/1,042,791) * 100 || [1042791.0, 948578.0, -9.034696310190633] || [948578.0, 1042791.0, 1042791.0] || True >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0
<< 17,845-17,202 || [17845.0, 17202.0, 643.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0
<< 1,320 - 1,086 || [1086.0, 1320.0, 234.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000baafcb
What was t

In [32]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.6751592356687898

In [33]:
len(annotated_results)

471